# Case Study - The Current

* The Current is an alternative radio station
* We will pull information about the play list.

# Step 0 - Inspect the following page

* Song title
* Artist
* Play time
* Day, date, period (am/pm)

http://www.thecurrent.org/playlist/2014-01-01/01

In [1]:
!pip install composable
!pip install composablesoup
!pip install composable --upgrade
!pip install composablesoup --upgrade

Requirement already up-to-date: composable in /home/sbada9048/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages (0.2.5)
Requirement already up-to-date: composablesoup in /home/sbada9048/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages (0.2.4)


In [2]:
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head
from composable.strict import map, filter
from composable.string import replace, split
from composable import from_toolz as tlz

In [20]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
r = s.get('http://www.thecurrent.org/playlist/2014-01-01/01')
current = BeautifulSoup(r.content, "html.parser")

# Step 1 - Pull off the period of the day (am/pm)

Pull out the "am"/"pm"

1. Inspect the element
2. Identify the html tag and class
3. Search the soup
    1. There should be one item returned
4. Use soup\string methods to pull out the info

In [21]:
(current
>> find('span', attrs = {'class':'hour-header'})
>> head(2)
)

In [22]:
from composable import pipeable
strip = pipeable(lambda s: s.strip())
(current
 >> find('span', attrs = {'class':'hour-header'})
 >> get_text
 >> strip
)

'1:00 am to  2:00 am'

In [23]:
strip = pipeable(lambda s: s.strip())
(current
 >> find('span', attrs = {'class':'hour-header'})
 >> get_text
 >> strip
 >> split(' ')
)

['1:00', 'am', 'to', '', '2:00', 'am']

In [24]:
strip = pipeable(lambda s: s.strip())
(current
 >> find('span', attrs = {'class':'hour-header'})
 >> get_text
 >> strip
 >> split(' ')
 >> tlz.get(1)
)

'am'

In [25]:
get_period = pipeable(lambda soup: soup
                     >> find('span', attrs = {'class':'hour-header'})
                     >> get_text
                     >> strip
                     >> split(' ')
                     >> tlz.get(1)
                     )
get_period(current)

'am'

# Step 2 - Pull off DJ

Use a similar process to pull off the DJ.


In [39]:
(current)

<!DOCTYPE html>

<html class="svg">
<head>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"4feae813d0","applicationID":"59426475","transactionName":"dVpcTRRZClhVEU1JWVdMXlAVQklHWAwV","queueTime":0,"applicationTime":296,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"4feae813d0",applicationID:"59426475"};window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var i=t[n]={exports:{}};e[n][0].call(i.exports,function(t){var i=e[n][1][t];return r(i||t)},i,i.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(e,t,n){function r(){}function i(e,t,n){return function(){return o(e,[u.now()].concat(c(arguments)),t?null:this,n),t?void 0:this}}var o=e("handle"),a=e(6),c=e(7),f=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof w

In [40]:
(current
 >> find('h5', attrs = {'class':'currentDj'})
)

<h5 class="currentDj">DJ: Mary Lucia</h5>

In [41]:
(current
 >> find('h5', attrs = {'class':'currentDj'})
 >> get_text
)

'DJ: Mary Lucia'

# Step 3 - Pull out the day of the week

* Pull out the day of the week

In [43]:
(current
 >> find('a', attrs = {'class':'start-picker'})
)

<a class="start-picker" href="#archives">Wednesday, Jan 01, 2014</a>

In [44]:
(current
 >> find('a', attrs = {'class':'start-picker'})
 >> get_text
)

'Wednesday, Jan 01, 2014'

In [ ]:
(current
 >> find('a', attrs = {'class':'start-picker'})
 >> get_text
)

In [45]:
(current
 >> find('a', attrs = {'class':'start-picker'})
 >> get_text
 >> split(' ')
)

['Wednesday,', 'Jan', '01,', '2014']

In [58]:
(current
 >> find('a', attrs = {'class':'start-picker'})
 >> get_text
 >> split(' ')
 >> tlz.get(0)
)

'Wednesday,'

# Title of each song

1. Inspect the element
2. Identify the html tag and class
3. Use `find_all` to make a list of all relevant tags
4. Pull off an example case
5. Write a function to pull out the title
6. Write a single pipe to convert the original soup into a list of titles. 
7. Verify you have the right number of titles.
8. Package the pipe in a function named `get_title`

In [ ]:
(current)

In [53]:
(current
 >> find_all('h5', attrs  = {'class':'title'})
)

[<h5 class="title">Holy Roller</h5>,
 <h5 class="title">Kingdom of Rust</h5>,
 <h5 class="title">Black Dog</h5>,
 <h5 class="title">Turn It Around</h5>,
 <h5 class="title">Flavor of the Month</h5>,
 <h5 class="title">Potential Wife</h5>,
 <h5 class="title">24 Hours</h5>,
 <h5 class="title">Who's Gonna Shoe Your Pretty Little Feet?</h5>,
 <h5 class="title">Marigold</h5>,
 <h5 class="title">High Road</h5>,
 <h5 class="title">The Vampyre Of Time and Memory</h5>,
 <h5 class="title">Valerie Plame</h5>,
 <h5 class="title">Morning Song</h5>,
 <h5 class="title">(You Will) Set The World On Fire</h5>,
 <h5 class="title">Sixteen Saltines</h5>,
 <h5 class="title">Wave of Mutilation</h5>]

In [82]:
example = (current
 >> find_all('h5', attrs  = {'class':'title'})
 >> tlz.get(0)
)
example

<h5 class="title">Holy Roller</h5>

In [89]:
pull_title = pipeable(lambda soup: soup.replace('<h5 class="title">', '').replace('</h5>',''))
pull_title(str(example))

'Holy Roller'

In [93]:
get_title = (current
             >> find_all('h5', attrs  = {'class':'title'})
             >> map(str)
             >> map(pull_title)
             )
get_title

['Holy Roller',
 'Kingdom of Rust',
 'Black Dog',
 'Turn It Around',
 'Flavor of the Month',
 'Potential Wife',
 '24 Hours',
 "Who's Gonna Shoe Your Pretty Little Feet?",
 'Marigold',
 'High Road',
 'The Vampyre Of Time and Memory',
 'Valerie Plame',
 'Morning Song',
 '(You Will) Set The World On Fire',
 'Sixteen Saltines',
 'Wave of Mutilation']

In [95]:
all = (current
 >> find_all('h5', attrs  = {'class':'title'})
 >> map(get_text)
)
all

['Holy Roller',
 'Kingdom of Rust',
 'Black Dog',
 'Turn It Around',
 'Flavor of the Month',
 'Potential Wife',
 '24 Hours',
 "Who's Gonna Shoe Your Pretty Little Feet?",
 'Marigold',
 'High Road',
 'The Vampyre Of Time and Memory',
 'Valerie Plame',
 'Morning Song',
 '(You Will) Set The World On Fire',
 'Sixteen Saltines',
 'Wave of Mutilation']

In [99]:
all = (current
 >> find_all('h5', attrs  = {'class':'title'})
 >> map(get_text)
)
all

['Holy Roller',
 'Kingdom of Rust',
 'Black Dog',
 'Turn It Around',
 'Flavor of the Month',
 'Potential Wife',
 '24 Hours',
 "Who's Gonna Shoe Your Pretty Little Feet?",
 'Marigold',
 'High Road',
 'The Vampyre Of Time and Memory',
 'Valerie Plame',
 'Morning Song',
 '(You Will) Set The World On Fire',
 'Sixteen Saltines',
 'Wave of Mutilation']

In [100]:
True if len(all) == 16 else False

True

In [102]:
def get_titles(page):
    """
    Args - 
        Page - String(The entire page)
    Returns - 
        A list with all the titles
    """
    pull_title = pipeable(lambda soup: soup.replace('<h5 class="title">', '').replace('</h5>',''))
    result =     (page
                  >> find_all('h5', attrs  = {'class':'title'})
                  >> map(str)
                  >> map(pull_title)
                  )
    return result
def test_get_titles():
    assert len(get_titles(current)) == 16
test_get_titles()

In [104]:
help(get_titles)

Help on function get_titles in module __main__:

get_titles(page)
    Args - 
        Page - String(The entire page)
    Returns - 
        A list with all the titles



# Pull off the name of the artist

1. Inspect the element
2. Identify the html tag and class
3. Use `find_all` to make a list of all relevant tags
4. Pull off an example case
5. Write a function to pull out the artist
6. Write a single pipe to convert the original soup into a list of artists. 
7. Verify you have the right number of artists.
8. Package the pipe in a function named `get_artist`


In [106]:
(current
 >> find_all('h5', attrs  = {'class':'artist'})
)

[<h5 class="artist">Thao and The Get Down Stay Down</h5>,
 <h5 class="artist">Doves</h5>,
 <h5 class="artist">Frankie Lee</h5>,
 <h5 class="artist">Lucius</h5>,
 <h5 class="artist">The Posies</h5>,
 <h5 class="artist">Strange Names</h5>,
 <h5 class="artist">Sky Ferreira</h5>,
 <h5 class="artist">Billie Joe and Norah</h5>,
 <h5 class="artist">J. Roddy Walston and The Business</h5>,
 <h5 class="artist">Cults</h5>,
 <h5 class="artist">Queens of the Stone Age</h5>,
 <h5 class="artist">The Decemberists</h5>,
 <h5 class="artist">The Avett Brothers</h5>,
 <h5 class="artist">David Bowie</h5>,
 <h5 class="artist">Jack White</h5>,
 <h5 class="artist">Pixies</h5>]

In [108]:
example = (current
 >> find_all('h5', attrs  = {'class':'artist'})
 >> tlz.get(0)
)
example

<h5 class="artist">Thao and The Get Down Stay Down</h5>

In [114]:
pull_artist = pipeable(lambda soup: soup.replace('<h5 class="artist">', '').replace('</h5>',''))
pull_artist(str(example))

'Thao and The Get Down Stay Down'

In [115]:
get_artist = (current
             >> find_all('h5', attrs  = {'class':'artist'})
             >> map(str)
             >> map(pull_artist)
             )
get_artist

['Thao and The Get Down Stay Down',
 'Doves',
 'Frankie Lee',
 'Lucius',
 'The Posies',
 'Strange Names',
 'Sky Ferreira',
 'Billie Joe and Norah',
 'J. Roddy Walston and The Business',
 'Cults',
 'Queens of the Stone Age',
 'The Decemberists',
 'The Avett Brothers',
 'David Bowie',
 'Jack White',
 'Pixies']

In [116]:
all = (current
             >> find_all('h5', attrs  = {'class':'artist'})
             >> map(get_text)
             )
all

['Thao and The Get Down Stay Down',
 'Doves',
 'Frankie Lee',
 'Lucius',
 'The Posies',
 'Strange Names',
 'Sky Ferreira',
 'Billie Joe and Norah',
 'J. Roddy Walston and The Business',
 'Cults',
 'Queens of the Stone Age',
 'The Decemberists',
 'The Avett Brothers',
 'David Bowie',
 'Jack White',
 'Pixies']

In [117]:
True if len(all) == 16 else False

True

In [122]:
def get_artists(page):
    """
    Args - 
        Page - String(The entire page)
    Returns - 
        A list with all the artists
    """
    pull_artist = pipeable(lambda soup: soup.replace('<h5 class="artist">', '').replace('</h5>',''))
    result =     (page
                  >> find_all('h5', attrs  = {'class':'artist'})
                  >> map(str)
                  >> map(pull_artist)
                  )
    return result
def test_get_artists():
    assert len(get_artists(current)) == 16
test_get_titles()

In [123]:
help(get_artists)

Help on function get_artists in module __main__:

get_artists(page)
    Args - 
        Page - String(The entire page)
    Returns - 
        A list with all the artists



In [124]:
get_artists(current)

['Thao and The Get Down Stay Down',
 'Doves',
 'Frankie Lee',
 'Lucius',
 'The Posies',
 'Strange Names',
 'Sky Ferreira',
 'Billie Joe and Norah',
 'J. Roddy Walston and The Business',
 'Cults',
 'Queens of the Stone Age',
 'The Decemberists',
 'The Avett Brothers',
 'David Bowie',
 'Jack White',
 'Pixies']